# AIRFOIL ANALYSIS NOTEBOOK

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import numpy as np

from ICARUS.Core.units import calc_mach, calc_reynolds
from ICARUS.Airfoils.airfoil import Airfoil
from ICARUS.Core.struct import Struct
from ICARUS.Computation.Solvers.solver import Solver
from ICARUS.Computation.Solvers.OpenFoam.files.setup_case import MeshType

# Load DB

In [3]:
from ICARUS.Database import DB
DB.load_data()

ERROR:root:Airfoil 0024 not found in DB2D or EXTERNAL DB


Airfoil 0024 not found in DB2D or EXTERNAL DB
Error in interpolating the airfoil data
Lower xs:
[0.000000e+00 6.870000e-05 7.181000e-04 1.878700e-03 3.843900e-03
 6.322100e-03 8.814300e-03 1.181410e-02 1.782690e-02 2.154460e-02
 2.756840e-02 3.359530e-02 3.962520e-02 4.565700e-02 5.169020e-02
 5.973630e-02 6.979530e-02 7.985710e-02 8.991900e-02 9.998280e-02
 1.100465e-01 1.251442e-01 1.402415e-01 1.553399e-01 1.704383e-01
 1.855357e-01 2.006342e-01 2.157315e-01 2.308300e-01 2.459284e-01
 2.610257e-01 2.761242e-01 2.912235e-01 3.113556e-01 3.314900e-01
 3.516258e-01 3.717628e-01 3.919010e-01 4.120401e-01 4.321793e-01
 4.523206e-01 4.724628e-01 4.926062e-01 5.127506e-01 5.328960e-01
 5.530424e-01 5.731888e-01 5.933372e-01 6.134868e-01 6.336375e-01
 6.537892e-01 6.739418e-01 6.940945e-01 7.142490e-01 7.344044e-01
 7.545605e-01 7.747174e-01 7.948748e-01 8.150318e-01 8.351902e-01
 8.553490e-01 8.755080e-01 8.956673e-01 9.158267e-01 9.259060e-01
 9.359855e-01 9.460661e-01 9.561460e-01 9.6622

In [4]:
print(f"Total number of loaded airfoils {len(list(DB.foils_db.airfoils.keys()))}")
print(f"Total number of computed airfoil data {len(list(DB.foils_db._data.keys()))}")
print(f"Total number of computed airfoil polars {len(list(DB.foils_db.polars.keys()))}")

Total number of loaded airfoils 1599
Total number of computed airfoil data 90
Total number of computed airfoil polars 90


# Reynolds And Mach and AoA

In [5]:
# Assumptions
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 35.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max
print(f"Mach range: {mach_min} - {mach_max}")
print(f"Mach:\n{mach}\n")

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
# reynolds = np.linspace(reynolds_min, reynolds_max, 10)
reynolds = [5e5, 1e6]

REYN = reynolds_max
print(f"Reynolds range: {reynolds_min} - {reynolds_max}")
print(f"Re:\n{reynolds}\n")
# ANGLE OF ATTACK SETUP
aoa_max: float = 12
aoa_min: float = -10
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

print(f"Angles of attack:\n{angles}\n")

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

Mach range: 0.029385836027034967 - 0.08815750808110491
Mach:
[0.08815751 0.08162732 0.07509714 0.06856695 0.06203676 0.05550658
 0.04897639 0.04244621 0.03591602 0.02938584]

Reynolds range: 32051.28205128205 - 1121794.8717948718
Re:
[500000.0, 1000000.0]

Angles of attack:
[-10.   -9.5  -9.   -8.5  -8.   -7.5  -7.   -6.5  -6.   -5.5  -5.   -4.5
  -4.   -3.5  -3.   -2.5  -2.   -1.5  -1.   -0.5   0.    0.5   1.    1.5
   2.    2.5   3.    3.5   4.    4.5   5.    5.5   6.    6.5   7.    7.5
   8.    8.5   9.    9.5  10.   10.5  11.   11.5  12. ]



In [6]:
cleaning = False
calcF2W = True
calcOpenFoam = False
calcXFoil = True

# Get Airfoils

In [7]:
all_airfoils = list(DB.foils_db.airfoils.keys())

airfoils_to_compute = [airfoil  for airfoil in all_airfoils if 
            (
                airfoil.upper().startswith('NACA0024') 
                # airfoil.upper().startswith('AG') or 
                # airfoil.upper().startswith('CLARK') or
                # airfoil.upper().startswith('DAE') or
                # airfoil.upper().startswith('E') or
                # airfoil.upper().startswith('H') or
                # airfoil.upper().startswith('M') or
                # airfoil.upper().startswith('N') or
                # airfoil.upper().startswith('O') or
                # airfoil.upper().startswith('W') 
            )
]
print(len(airfoils_to_compute[:2]))


1


# Xfoil

In [8]:
airf: Airfoil = DB.get_airfoil(airfoils_to_compute[0])
airf.plot()
airf.name

'NACA0024'

In [12]:
if calcXFoil:
    from ICARUS.Computation.Solvers.Xfoil.xfoil import Xfoil 

    for airfoil in airfoils_to_compute[:2]:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get airfoil
        airf: Airfoil = DB.foils_db.airfoils[airfoil]
        airf.plot(scatter=True)
        # airf.repanel(120, distribution="cosine")
        xfoil: Solver = Xfoil()

        # Import Analysis
        analysis: str = xfoil.get_analyses_names()[1]  # Run
        xfoil.select_analysis(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=False)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.mach = MACH
        options.reynolds = reynolds
        options.min_aoa = aoa_min
        options.max_aoa = aoa_max
        options.aoa_step = 0.5

        # Set Solver Options
        solver_parameters.max_iter = 100
        solver_parameters.Ncrit = 9
        solver_parameters.xtr = (1, 1)
        solver_parameters.print = False
        
        xfoil.define_analysis(options, solver_parameters)
        xfoil.print_analysis_options()
    
        # RUN
        xfoil.execute(parallel=False)
        # Get polar
        polar = DB.foils_db.get_polars(airf.name.upper(), "Xfoil")
        polar.plot()


Running airfoil NACA0024

Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                       | Description                       |
|-----------|-----------------------------|-----------------------------------|
| airfoil   | Complex Datatype (NACA0024) | Airfoil Object                    |
| mach      | Complex Datatype            | Mach number                       |
| reynolds  | Complex Datatype            | List of Reynold's numbers to run  |
| min_aoa   | -10                         | Minimum angle of attack           |
| max_aoa   | 12                          | Maximum angle of attack           |
| aoa_step  | 0.5                         | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a m

  0%|          | 0/2 [00:00<?, ?it/s]

Adding NACA0024 to the database
Analysis Completed


In [10]:
DB.foils_db.polars

Struct(AG03=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED648BC50>), AG04=Struct(), AG08=Struct(), AG09=Struct(), AG10=Struct(), AG11=Struct(), AG12=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED649BA40>), AG13=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED64C5D30>), AG14=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED64C6960>), AG16=Struct(), AG17=Struct(), AG18=Struct(), AG19=Struct(), AG23=Struct(), AG24=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED64EC890>), AG25=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED64EE210>), AG26=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED6509FA0>), AG27=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED650BE60>), AG35=Struct(), AG36=Struct(), AG37=Struct(Xfoil=<ICARUS.Airfoils.airfoil_polars.Polars object at 0x000001BED651D8E0>), AG38=Struct(

# Foil2Wake

In [11]:
if calcF2W :
    # Import Solver
    from ICARUS.Computation.Solvers.Foil2Wake.f2w_section import Foil2Wake
    f2w_s: Solver = Foil2Wake()

    for airfoil in airfoils_to_compute[:1]:
        print(f"\nRunning airfoil {airfoil}\n")
        # Import Analysis
        analysis: str = f2w_s.get_analyses_names()[1]  # Run
        f2w_s.select_analysis(analysis)

        # Get Options
        options: Struct = f2w_s.get_analysis_options(verbose=False)
        solver_parameters: Struct = f2w_s.get_solver_parameters()

        # Set Options
        options.airfoil = DB.foils_db.airfoils[airfoil]
        options.reynolds = reynolds
        options.mach = MACH
        options.angles = angles

        solver_parameters.f_trip_upper = 0.03
        solver_parameters.f_trip_low = 0.1

        # RUN
        f2w_s.define_analysis(options, solver_parameters)
        f2w_s.print_analysis_options()
        f2w_s.execute(parallel=True)

        # Get Results
        polars = f2w_s.get_results()



Running airfoil NACA0024

Airfoil Polar Analysis for Multiple Reynold's Numbers
Available Options of Foil2Wake for Airfoil Polar Analysis for Multiple Reynold's Numbers: 

| VarName   | Value                       | Description                      |
|-----------|-----------------------------|----------------------------------|
| airfoil   | Complex Datatype (NACA0024) | Airfoil Object                   |
| mach      | Complex Datatype            | Mach number                      |
| reynolds  | Complex Datatype            | List of Reynold's numbers to run |
| angles    | Complex Datatype            | List of angles to run polars     |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver Foil2Wake:
	Analysis Airfoil Polar Analysis for Multiple Reynold's Numbers...


		5.000e+05-pos-0.0 Progress::   0%|                              | 0/251 [00:00<?, ?it/s]

		5.000e+05-neg-0.0 Progress::   0%|                              | 0/251 [00:00<?, ?it/s]

		1.000e+06-pos-0.0 Progress::   0%|                              | 0/251 [00:00<?, ?it/s]

		1.000e+06-neg-0.0 Progress::   0%|                              | 0/251 [00:00<?, ?it/s]

Analysis Finished 1.000e+06 9.0
Analysis Finished 1.000e+06 10.0


: 

In [ ]:
print(airfoils_to_compute[1])
DB.foils_db.polars[airfoils_to_compute[1]]

# Mapflow

In [ ]:
# Not Implemented

# OpenFoam (NOT HIGH MACH)

In [ ]:
if calcOpenFoam:
    airfoils_to_compute: list[str] = [airf.name]#["0008"]
    from ICARUS.Computation.Solvers.OpenFoam.open_foam import OpenFoam

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plot()
        open_foam: Solver = OpenFoam()

        # Import Analysis
        analysis: str = open_foam.get_analyses_names()[0]  # Run
        open_foam.select_analysis(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.angles = angles
        options.reynolds = REYN
        options.mach = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type = MeshType.structAirfoilMesher
        solver_parameters.max_iterations = 20
        solver_parameters.silent = False
        
        # RUN
        open_foam.set_analysis_options(options)
        open_foam.print_analysis_options()
        open_foam.execute()